# Chapter 3: Functional Programming

* Functional Programming: Declarative programming philosophy
    * Tell a computer what you want
    * Characteristics:
        * Pure functions
            * Return the same result given a set of input args
        * First class and high-order functions
        * Immutability
    * EX: SQL
* Procedural Programming: Explanatory programming philosophy
    * Tell a computer how to do something
    * EX: Python
    
```
# main.tf
terraform {
  required_version = ">= 0.15"
  required_providers {
    random = {
      source  = "hashicorp/random"
      version = "~> 3.0"
        }
    }
}

```

## Program Flow
1) Instantiate the terraform settings block
2) Assign the values with a variable definition file
3) Define the relationships of the hash map
4) Check for variable validation (this would evalidate to false)
5) Creates a shuffled resource for each variable 
6) Output this to a templated file

    
## Using Variables
* Input variables: (variable "words"{ .. })
    * User-supplied values that parametrize Terraform modules
    * Does not alter the source code
    * Input args:
        * default: Variable is mandatory and must be set
        * description: String value for documentation
        * type: Type constraint
        * Validation: Nested block of validation rules

```
# main.tf
variable "words" {
  description = "A word pool to use for Mad Libs"
  type = object({
    nouns      = list(string),
    adjectives = list(string),
    verbs = list(string),
    })

  validation {
    condition = length(var.words['noun']) >= 20
    error_message = "At least 20 nouns must be supplied."
    }
}

resource "random_shuffle" "random_nouns" {
  input = var.words["nouns"]
}

resource "random_shuffle" "random_adjectives" {
  input = var.words["adjectives"]
}

resource "random_shuffle" "random_verbs" {
  input = var.words["verbs"]
}


-----
# terraform.tfvars
words = {
nouns = ["army", "panther", "walnuts"]
adjectives = ["bitter", "sticky", "thundering"]
verbs = ["run", "dance", "love", "respect", "kicked", "baked"]
}
```


### Using Functions
* Terraform functions: (templatefile ('path/*.txt', '{template variables})
    * Transform inputs into outputs
    * NO USER-DEFINED functions
        * There's a lot in TF, but you are restricted to ~100
        * Functionality is extended by writing providers not functions
        
```
# main.tf
templatefile('${path.module}/.../, 
{
    nouns = random_shuffle.random_nouns.result
    adjectives = random_shuffle.random_adjectives.result
    verbs = random_shuffle.random_verbs.result
}
```

### Using Outputs / Templates
* Output values (output name{...})
    * Passes values between modules
    * Prints values to the CLI

```
# main.tf

output "mad_libs" {
  value = templatefile("${path.module}/templates/alice.txt",
    {
      nouns      = random_shuffle.random_nouns.result
      adjectives = random_shuffle.random_adjectives.result
}) }

---
alice.txt
 Lewis Carroll's classic, "Alice's Adventures in Wonderland", as well
        as its ${adjectives[0]} sequel, "Through the Looking ${nouns[0]}",
        have enchanted both the young and old ${nouns[1]}s for the last
        ... end when Alice awakens from her ${nouns[8]}.
        
```



In [ ]:
* 